<a href="https://colab.research.google.com/github/maxfloresv/FM849/blob/feat%2Fupdate-course-tasks/20261/tasks/02_regression_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FM849 - Programación Científica para Proyectos de Inteligencia Artificial (IA)

**Antes de empezar, asegúrense de hacer una copia de este archivo en su cuenta de Google (lo pueden hacer desde la barra de opciones de arriba). Si editan directamente la tarea desde este enlace, no se guardará.**

## Tarea 2: Regresión y clasificación

👨‍🏫 **Profesores de cátedra**

- Máximo Flores Valenzuela ($\texttt{mflores@dcc.uchile.cl}$).
- Héctor Jiménez Orellana ($\texttt{hector.jimenezor@gmail.com}$).

👥 **Equipo de trabajo**

- Completar acá con la lista de estudiantes. **Importante**: trabajos sin nombres no serán evaluados.

---

La evaluación se hará con la escala chilena del 1.0 al 7.0, con lo cual, la tarea tiene como máximo 6 puntos alcanzables. Cada sección tiene su puntaje asignado en su título. **Sea ordenado en su desarrollo. Hay espacios demarcados en toda la tarea donde deben ir sus respuestas. Tareas desordenadas tendrán un descuento de hasta 1 punto.**

Esta tarea está sujeta a las reglas mencionadas en la [introducción del curso](https://github.com/maxfloresv/FM849/blob/master/20261/00_intro.pdf). Asegúrense de tenerlas claras antes de desarrollarla.

# Introducción

En esta tarea, nos enfocaremos en **regresión y clasificación**. A continuación, se realizan las importaciones e instalaciones necesarias para trabajar. No las borre. Lo que puede hacer es extenderlas si necesita usar más herramientas.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
%%capture

!pip install ucimlrepo

# 1. 🚲 La ciudad inteligente y el guardián de las bicicletas $\textbf{(3 pts.)}$

La ciudad de Seúl está implementando un ambicioso proyecto de ciudad inteligente. Como parte de esta iniciativa, se ha desplegado un sistema de arriendo de bicicletas públicas que miles de personas utilizan diariamente para trasladarse al trabajo, la universidad y otros destinos.

Para que el sistema funcione correctamente, la municipalidad necesita anticiparse a la demanda respondiendo preguntas como:

- **¿Cuántas bicicletas se necesitarán mañana?**

- **¿Influye el clima en la cantidad de arriendos?**

- **¿Es distinto el comportamiento entre días laborales y fines de semana?**

Para responder estas preguntas, se ha creado un asistente inteligente llamado BikeMind 🤖, cuyo objetivo es predecir cuántas bicicletas serán rentadas en un día y hora determinado, usando información como la temperatura, la humedad, el viento y otras variables ambientales.

Tu misión como parte del equipo de datos es enseñar a BikeMind 🤖 a aprender desde datos históricos, utilizando técnicas de aprendizaje supervisado, comenzando por uno de los modelos más clásicos y fundamentales: **la regresión lineal**.

> Antes de empezar, se le recomienda revisar los siguientes recursos de las cátedras:
> - [Clase 10: Regresión lineal y regularización](https://github.com/maxfloresv/FM849/blob/master/20261/10_lr_regularization.pdf).
> - [Tutorial de regresión lineal y no lineal](https://colab.research.google.com/drive/1lXo30brhnMfGAgkqM814eilnf4rMR4Nu?usp=sharing).

![](https://dhj3y7nsb2ne02la.public.blob.vercel-storage.com/blog/seoul-bike-tours-ZmWesFGQUgMVqqG1ZURNroPzVVp5yN.webp)

## Carga del conjunto de datos

En esta sección, cargaremos el conjunto de datos. Este consiste en un objeto de tipo `pd.DataFrame` con $8.760$ filas. Cada fila representa un dato asociado a un día-hora determinado donde se obtuvieron la cantidad de bicicletas rentadas.

La variable objetivo a predecir es `Bike Rented Count`, que representa el número de bicicletas rentadas, mientras que el resto de las columnas se utilizarán como variables de entrada para entrenar el modelo encargado de estimar la cantidad de bicicletas.

In [ ]:
BIKE_DATA_URL = "https://archive.ics.uci.edu/ml/machine-learning-databases/00560/SeoulBikeData.csv"
bike_df = pd.read_csv(BIKE_DATA_URL, encoding='unicode_escape')

In [ ]:
# Información de los datos
bike_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Date                       8760 non-null   object 
 1   Rented Bike Count          8760 non-null   int64  
 2   Hour                       8760 non-null   int64  
 3   Temperature(°C)            8760 non-null   float64
 4   Humidity(%)                8760 non-null   int64  
 5   Wind speed (m/s)           8760 non-null   float64
 6   Visibility (10m)           8760 non-null   int64  
 7   Dew point temperature(°C)  8760 non-null   float64
 8   Solar Radiation (MJ/m2)    8760 non-null   float64
 9   Rainfall(mm)               8760 non-null   float64
 10  Snowfall (cm)              8760 non-null   float64
 11  Seasons                    8760 non-null   object 
 12  Holiday                    8760 non-null   object 
 13  Functioning Day            8760 non-null   objec

## 1.1. Selección y conversión de _inputs_ $\textrm{(0,2 pts.)}$

Para esta tarea, consideraremos las siguientes columnas del conjunto de datos como _inputs_: `Hour`, `Temperature(°C)`, `Humidity(%)`, `Wind speed (m/s)`, `Visibility (10m)`, `Dew point temperature(°C)`, `Solar Radiation (MJ/m2)`, `Rainfall(mm)`, `Snowfall (cm)`, `Seasons` y `Holiday`. Asegúrese de entender bien su significado. Para ello, puede consultar [este enlace](https://archive.ics.uci.edu/dataset/560/seoul+bike+sharing+demand).

In [ ]:
inputs = [
  'Hour',
  'Temperature(°C)',
  'Humidity(%)',
  'Wind speed (m/s)',
  'Visibility (10m)',
  'Dew point temperature(°C)',
  'Solar Radiation (MJ/m2)',
  'Rainfall(mm)',
  'Snowfall (cm)',
  'Seasons',
  'Holiday'
]

# Datos de entrada del modelo
X = bike_df[inputs]
# Variable a predecir
y = bike_df[["Rented Bike Count"]]

Las columnas `Seasons` y `Holiday` correponden a variables categóricas. Convertiremos estas columnas a valores binarios (0 o 1) utilizando el método de *One-Hot Encoding*. Para esto, se usara el método `pd.get_dummies`.

In [ ]:
X = pd.get_dummies(
  data=X,
  columns=["Seasons", "Holiday"],
  dtype=int,
)

- Describa brevemente con sus propias palabras qué procedimiento realiza la función `pd.get_dummies` y por qué es importante en el contexto de un modelo de regresión lineal.

> **Respuesta**: _complete acá con su respuesta._

## 1.2. _Holdout_ $\textrm{(0,2 pts.)}$

- Describa con sus propias palabras en qué consiste la técnica de _holdout_ y por qué es importante en Machine Learning. $\textrm{(0,1 pts.)}$

> **Respuesta**: _complete acá con su respuesta._

- Implemente esta técnica en código en la celda de abajo. Use la función `train_test_split` para dividir los datos (`X` e `y`) en conjunto de entrenamiento y prueba. $\textrm{(0,1 pts.)}$

In [ ]:
# Complete con su código en esta celda.
from sklearn.model_selection import train_test_split

## 1.3. Escalamiento de los datos $\textrm{(0,6 pts.)}$

- Describa brevemente las variables seleccionadas del conjunto de datos. ¿De qué se trata cada una? ¿Son numéricas/categóricas? ¿Cuáles son los rangos de valores que pueden tomar, en el caso de ser numéricas? $\textrm{(0,3 pts.)}$

> **Respuesta**: _complete acá con su respuesta._

Para este ítem, **todas las variables numéricas deben ser escaladas (transformadas) utilizando `StandardScaler`** de Scikit-Learn. `StandardScaler` transforma los datos para que cada variable tenga media $\mu$ igual a 0 y desviación estándar $\sigma$ igual a 1.

- Justifique, con la descripción que hizo en el ítem anterior, por qué existe la necesidad de escalar las variables. Apóyese de la [documentación de Scikit-Learn](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html). $\textrm{(0,1 pts.)}$

> **Respuesta**: _complete acá con su respuesta._

- Implemente el escalador  `StandardScaler` en código en la celda de abajo, evitando el problema de _Data Leakage_. Para esto, solo ajuste con `fit_transform` el escalador en los datos de entrenamiento (este método permite hacer el ajuste y la transformación en la misma instrucción). En el conjunto de prueba, debe usar únicamente `transform`. $\textrm{(0,2 pts.)}$

_Indicación:_ Recuerde que si usamos información proveniente de los datos de prueba para hacer cualquier ajuste, estaríamos "filtrando" información que no queremos que el modelo sepa, porque estos datos, como no se han visto antes, sirven para evaluar el rendimiento.

In [ ]:
# Complete con su código en esta celda.
from sklearn.preprocessing import StandardScaler

## 1.4. Modelo de regresión lineal $\textrm{(1 pt.)}$

En esta sección, Ud. deberá entrenar un modelo de regresión lineal usando los datos de entrenamiento que extrajo anteriormente. Puede usar modelos ya implementados en Scikit-Learn, pero debe justificar todas las decisiones de diseño que no hayan sido mencionadas en los ítems anteriores.

Se recomienda utilizar la clase [`LinearRegression` de Scikit-Learn](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html), que vimos en las cátedras pasadas. Recuerde usar el metodo `fit` para ajustar su modelo lineal sobre el conjunto de datos de entrenamiento. $\textrm{(0,2 pts.)}$

In [ ]:
# Complete con su código en esta celda.
from sklearn.linear_model import LinearRegression

Luego del entrenamiento, **identifique la forma matemática que tiene su modelo**. Recuerde que la forma general de un modelo de regresión lineal es la siguiente:

$$
y = \beta_0 + \sum_{i=1}^n \beta_i x_i + \varepsilon
$$

donde $x_1, \dots, x_n$ son las variables usadas para realizar la predicción, $y$ es la variable a predecir, $\beta_0, \beta_1, \dots, \beta_n$ son los coeficientes que deben ser calculados por el modelo, y $\varepsilon$ es el ruido aleatorio que no podemos medir. $\textrm{(0,4 pts.)}$

> **Respuesta**: *complete acá con su respuesta.*

_Indicación_: Note que las variables predictoras $x_1, \dots, x_n$ tienen un orden que no es fortuito. Revise el orden de las variables con el atributo `feature_names_in_`. Por ejemplo, si su modelo lo instanció con `lr = LinearRegression()`, luego del `fit`, puede usar `lr.feature_names_in_`. Este atributo entrega las variables predictoras exactamente en el orden que debería usar para identificar la forma matemática del modelo.

- Con la forma matemática ya identificada, revise con `coef_` e `intercept_` los valores de los coeficientes asociados a cada variable predictora, y el intercepto con el eje $Y$ respectivamente. Explique cada coeficiente en el contexto del problema. **¿Tienen sentido los valores calculados por el modelo?** $\textrm{(0,4 pts.)}$

> **Respuesta**: *complete acá con su respuesta.*


## 1.5. Evaluación del desempeño del modelo $\textrm{(1 pt.)}$

- Investigue acerca de las métricas [MAE](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_absolute_error.html), [MSE](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) y [$R^2$](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html). ¿Para qué sirven en el contexto de las regresiones lineales? ¿Cómo se calculan? No es necesario que de la fórmula explícita, puede también dar una explicación intuitiva. $\textrm{(0,2 pts.)}$

> **Respuesta**: *complete acá con su respuesta.*

- Use el modelo entrenado en la subsección anterior (**1.4.**) para evaluar su rendimiento en el conjunto de prueba. Para esto, calcule las métricas MAE, MSE y $R^2$ comparando las predicciones generadas por el modelo con los valores reales de la cantidad de bicicletas rentadas en el conjunto de prueba. Complete con su código en la celda de abajo. $\textrm{(0,3 pts.)}$

In [ ]:
# Complete con su código en esta celda.

- Explique las métricas. ¿Cómo es el desempeño de su modelo? ¿Cómo cree que podría mejorarlo? Desarrolle sus ideas con holgura. Mientras más valor tenga su análisis para el problema, más puntaje se asignará. $\textrm{(0,5 pts.)}$

> **Respuesta**: _complete acá con su respuesta._

# 2. 🌍 Traduciendo sólo con diccionarios $\textbf{(3 pts.)}$

![](https://media3.giphy.com/media/v1.Y2lkPTZjMDliOTUyZjg3bmkxcDFycGpubW5jZjlibmM5dmJkOHFkMWhmNnBhdnJzb2N4dSZlcD12MV9naWZzX3NlYXJjaCZjdD1n/FVZoYkTx3cuVCkEavD/200.gif)

Ud. se encuentra en Alemania, pero enfrenta una situación inesperada: **no tiene su celular**, por lo que no puede utilizar aplicaciones de traducción automática 😅.

Durante su viaje, Ud. debe comunicarse con personas en diferentes situaciones: en una tienda, en una estación de tren, o en la calle. Algunas personas se comunican en alemán, mientras que otras lo hacen en inglés, ya que ambos idiomas son relativamente comunes en el país.

En un intento por entender lo que le dicen las personas, Ud. decide comprar dos diccionarios: uno de alemán y otro de inglés. El problema es que Ud. no reconoce de inmediato en qué idioma le están hablando. Sin esta información, no sabe cuál de sus diccionarios utilizar para poder entender el mensaje.

Usar el diccionario incorrecto no sirve, ya que una palabra alemana no aparecerá en el diccionario de inglés y viceversa. Por esta razón, identificar el idioma es el primer paso esencial para poder comunicarse.

Para resolver este problema, Ud. decide implementar un sistema de inteligencia artificial que le permita identificar el idioma en el que le están hablando. Para esto, asuma que trabajará con una sóla palabra a la vez.

- El objetivo será que el sistema reciba una palabra como entrada y devuelva una predicción indicando si la palabra pertenece al idioma inglés o alemán.

Por ejemplo, la palabra `house` debería ser identificada como **inglés**, y la palabra `rufen` debería ser identificada como **alemán**.

## 🧠 ¿Cómo puede resolver el problema?

Para resolver este problema, Ud. dispone de un conjunto de datos etiquetados, donde cada fila contiene dos columnas con:
- Una **palabra**.
- Una etiqueta asociada a la palabra que indica su **idioma** (inglés o alemán).

Este tipo de problema corresponde a un caso de aprendizaje supervisado, ya que el modelo aprende a partir de ejemplos donde la respuesta correcta es conocida.

El desafío consiste en entrenar un modelo que aprenda patrones en las letras de las palabras, tales como combinaciones frecuentes de caracteres, para identificar el idioma incluso en palabras que nunca ha visto antes.

> ⚠️ **Nota importante**: No es necesario conocer inglés ni alemán para resolver esta tarea. Desde el punto de vista del modelo, las palabras son simplemente secuencias de letras, y el aprendizaje se basa en detectar regularidades en esas secuencias.

## Carga del conjunto de datos

En este problema, los conjuntos de datos de entrenamiento y prueba ya están generados. Utilizaremos el conjunto de datos de un [curso de Machine Learning de la Universidad de Cornell](https://github.com/gabrielesavethekiwi/CS-5780---Intro-to-Machine-Learning-/tree/main/CS5780/P3). Ud. no debe preocuparse por los datos, estos serán cargados en los siguientes bloques de código.

In [ ]:
import pandas as pd

data_entrenamiento = pd.read_csv("https://raw.githubusercontent.com/hectorjimenez12/curso-programacion-para-IA-edv/refs/heads/main/data_train_eng_ger.csv")
data_prueba = pd.read_csv("https://raw.githubusercontent.com/hectorjimenez12/curso-programacion-para-IA-edv/refs/heads/main/data_test_eng_ger.csv")

data_entrenamiento = data_entrenamiento.sample(frac=1).reset_index(drop=True)
data_prueba = data_prueba.sample(frac=1).reset_index(drop=True)

In [ ]:
print("Algunos ejemplos aleatorios:")
data_entrenamiento.sample(10)

Algunos ejemplos aleatorios:


,Palabra,Idioma
939,musiker,Aleman
381,vaterunser,Aleman
797,mehlsaecken,Aleman
1320,indicator,Ingles
730,hauch,Aleman
891,hatten,Aleman
1553,reading,Ingles
815,offenbar,Aleman
632,picture,Ingles
1436,hinausschreien,Aleman


Ahora, separaremos el _DataFrame_ en datos de entrada para su modelo (`X_entrenamiento` y `X_prueba`) y respuestas esperadas (etiquetas `y_entrenamiento` y `y_prueba`).  

In [ ]:
X_entrenamiento = data_entrenamiento["Palabra"]
y_entrenamiento = data_entrenamiento["Idioma"]

X_prueba = data_prueba["Palabra"]
y_prueba = data_prueba["Idioma"]

Finalmente, recordemos que los modelos sólo trabajan con números. Por esto, convertiremos las respuestas esperadas (`y_entrenamiento` y `y_prueba`) a valores enteros, donde inglés será $0$ y alemán $1$.

In [ ]:
y_entrenamiento = [1 if idioma == "Aleman" else 0 for idioma in y_entrenamiento]
y_prueba = [1 if idioma == "Aleman" else 0 for idioma in y_prueba]

## 2.1. Generación de _inputs_ $\textrm{(1 pt.)}$

El _DataFrame_ utilizado no contiene datos numéricos, más bien, son sólo datos en formato texto (palabra e idioma). Es por esto que es necesario generar nuevos datos de entrada (_inputs_). Ud. deberá generar datos de entrada numéricos a partir de la columna `Palabra` (en `X_entrenamiento` y `X_prueba`) considerando la técnica de _One-Hot Encoding_.

Se utilizará un caso especial de _One-Hot Encoding_ conocido como _Bag of Characters_. Esto permite convertir una palabra a una lista de ceros y unos. Las columnas con $1$ indicaran que una letra está presente en la palabra, mientras que las columnas con $0$ indicarán que una letra no está presente en la palabra.

### Ejemplo de _Bag of Characters_

Tenemos dos nombres: Ana y Luis. El conjunto de letras observadas es el siguiente (no se consideran repeticiones):
```
{a, n, l, u, i, s}
```
Con esta información, la representación numérica de los nombres es:

| Nombre | a | n | l | u | i | s |
|------|---|---|---|---|---|---|
| Ana  | 1 | 1 | 0 | 0 | 0 | 0 |
| Luis | 0 | 0 | 1 | 1 | 1 | 1 |

donde, como fue mencionado, la tabla contiene un $1$ en una celda si para la fila que corresponda, la letra indicada en la columna aparece en el nombre, y $0$ si no.

Esta representación transforma texto en números y permite entrenar modelos de clasificación. Debe pensar que ahora cada columna *a, n, l, u, i, s* será un dato de entrada al modelo.

- Genere un código que le permita crear una representacion _Bag of Characters_ sobre la columna `Palabra` usando la funcion `CountVectorizer` de Scikit-Learn. Puede revisar su funcionamiento en el [siguiente enlace](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html). $\textrm{(0,8 pts.)}$

_Indicación_: Genere un objeto de tipo `CountVectorizer`. Luego, ajuste este objeto con el metodo `fit_transform` sobre las palabras del conjunto de entrenamiento. Finalmente, use este objeto para transformar las palabras del coljunto de prueba en una representacion _One-Hot Encoding_.

In [ ]:
# Complete el código para generar el Bag of Characters.
from sklearn.feature_extraction.text import CountVectorizer

- ¿Qué dimensiones tiene el conjunto de datos de entrada para el modelo? ¿Qué significan estas dimensiones? $\textrm{(0,2 pts.)}$

> **Respuesta**: _complete acá con su respuesta._

## 2.2. Ajuste de un modelo de clasificación $\textrm{(1 pt.)}$

Ud. debe seleccionar su modelo de clasificación **favorito** (sólo 1) y entrenarlo considerando como datos de entrada los _inputs_ generados en la seccion **2.1.**. Recuerde que la variable a predecir es el idioma de la palabra.

Se recomienda seleccionar un modelo tipo $K$ _Nearest Neighbors_, _Support Vector Machine_, _Naïve_-Bayes, _Decision Tree_, _Random Forest_ u otro visto en cátedras. Algunas clases de Scikit-Learn asociadas a estos modelos se enlistan a continuación:

- [$K$ _Nearest Neighbors_](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)
- [_Support Vector Machine_](https://scikit-learn.org/stable/modules/svm.html)
- [_Naïve_-Bayes](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html#sklearn.naive_bayes.MultinomialNB)
- [_Decision Tree_](https://scikit-learn.org/stable/modules/tree.html)
- [_Random Forest_](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)

Investigue sobre su modelo favorito y úselo para clasificar palabras.

In [ ]:
# Complete acá su código para generar el entrenamiento del modelo de clasificación.

## 2.3. Evaluación del modelo de clasificación $\textrm{(1 pt.)}$

- En esta parte, Ud. deberá evaluar el modelo de clasificación entrenado en la sección **2.2.** utilizando las metricas *Accuracy* y $\text{F1}$ *Score*. Calcule las métricas considerando el conjunto de entrenamiento y prueba por separado. $\textrm{(0,5 pts.)}$

In [ ]:
# Complete el código para calcular métricas de rendimiento en el modelo entrenado.

- ¿Cómo son los valores de *Accuracy* y $\text{F1}$ *Score* en los datos de entrenamiento y prueba? ¿Qué significan las diferencias entre estos valores? ¿Se le ocurre alguna mejora que pueda implementar en su modelo? $\textrm{(0,5 pts.)}$

> **Respuesta**: _complete acá con su respuesta._